<a href="https://colab.research.google.com/github/vilsonrodrigues/ComputerVision/blob/master/ITA_Challenge_2021-TrafficLightDetection/TrafficLightDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Baseado em: 

https://github.com/AlexeyAB/darknet

https://medium.com/analytics-vidhya/train-a-custom-yolov4-object-detector-using-google-colab-61a659d4868

## Step 0 - Criar Pastas no Drive

Monte o seu Drive e em seguida crie uma pasta para o seu projeto. Veja que antes eu criei uma pasta mais geral para os projetos de detecção

In [1]:
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!mkdir /content/drive/MyDrive/detection

mkdir: cannot create directory ‘/content/drive/MyDrive/detection’: File exists


In [3]:
!mkdir /content/drive/MyDrive/detection/TrafficLightDetection

## Step 1 - Clonando o Repositório Darknet 

Baixando o repositório que contêm a implementação da YOLOv4 e inserindo na pasta

In [4]:
%cd /content/drive/MyDrive/detection/TrafficLightDetection
!git clone https://github.com/AlexeyAB/darknet

/content/drive/MyDrive/detection/TrafficLightDetection
Cloning into 'darknet'...
remote: Enumerating objects: 15050, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 15050 (delta 16), reused 34 (delta 14), pack-reused 15002
Receiving objects: 100% (15050/15050), 13.48 MiB | 7.98 MiB/s, done.
Resolving deltas: 100% (10215/10215), done.
Checking out files: 100% (2028/2028), done.


Chegando se tudo ocorreu bem

In [5]:
!ls /content/drive/MyDrive/detection/TrafficLightDetection

darknet


## Step 2 - Montagem da Estrutura do Projeto

Usando o software LabelImg você pode etiquetar suas imagens para a detecção. A criação de um bom dataset é a parte mais fundamental em um modelo, se você criar um dataset mal anotado/enviesado o modelo vai aprender esse viés e quando for a produção vai performar mal

O dataset foi coletado por mim mesmo em uma disciplina num semestre atrás, ele contêm as 5 vogais em libras

As imagens precisam ser quadradas e o tamanho ser múltiplo de 32. As do dataset tem uma dimensão 1280x720x3

Copiando a pasta das imagens para a pasta do projeto

In [6]:
!cp -r -n /content/drive/MyDrive/Datasets/TrafficLightDetection/data /content/drive/MyDrive/detection/TrafficLightDetection/

Baixando os arquivos necessários

In [ ]:
%%capture
%cd /content/drive/MyDrive/detection/TrafficLightDetection
!wget -P data/ https://github.com/vilsonrodrigues/ComputerVision/blob/master/ITA_Challenge_2021-TrafficLightDetection/obj.names
!wget -P data/ https://github.com/vilsonrodrigues/ComputerVision/blob/master/ITA_Challenge_2021-TrafficLightDetection/obj.data 
!wget -P data/dev/ https://raw.githubusercontent.com/vilsonrodrigues/ComputerVision/master/ITA_Challenge_2021-TrafficLightDetection/dev.txt
!wget -P data/train/ raw.githubusercontent.com/vilsonrodrigues/ComputerVision/master/ITA_Challenge_2021-TrafficLightDetection/train.txt

O arquivo `obj.names` são as nossas classes, o arquivo é gerado automaticamente pelo LabelImg. Contêm as 5 vogais



---

O `obj.data` informa o número de classes, a localização do txt com a lista de imagens de treino e dev, a localização do arquivo .coco e a pasta onde vai ser o backup




---



O arquivo `yolov4.cfg` é onde está a estrutura da rede

`Width` e `Height` foram definidos como iguais a 224

O parâmetro `max_batches` é definido como num_classes * 2000 = 10000

`Steps` é dado por 2 parâmetros, 80% e 90% de `max_batches`, então vai dar 8000 e 9000, respectivamente

Em cada uma das camadas de convolução pré [yolo] tem (num_classes + 5) * 3 = 30 filtros convoluciontais. Em [yolo] foi configurado para ter as 5 classes do nosso dataset em vez das 80 tradicionais vindas no YOLO

In [8]:
!ls /content/drive/MyDrive/detection/TrafficLightDetection

darknet  data


In [9]:
!ls /content/drive/MyDrive/detection/TrafficLightDetection/data/

dev  obj.data  obj.names  train


Primeiro vamos mover a pasta para dentro do diretório para facilitar a execução


In [19]:
%cd /content/drive/MyDrive/detection/TrafficLightDetection/
!cp -r -n data/ darknet/

/content/drive/MyDrive/detection/TrafficLightDetection


In [20]:
%cd /content/drive/MyDrive/detection/TrafficLightDetection/
!ls

/content/drive/MyDrive/detection/TrafficLightDetection
darknet  data


In [21]:
%cd /content/drive/MyDrive/detection/TrafficLightDetection/darknet/data
!ls

/content/drive/MyDrive/detection/TrafficLightDetection/darknet/data
9k.tree     dev		 goal.txt		   labels	     person.jpg
coco9k.map  dog.jpg	 horses.jpg		   obj.data	     scream.jpg
coco.names  eagle.jpg	 imagenet.labels.list	   obj.names	     train
data	    giraffe.jpg  imagenet.shortnames.list  openimages.names  voc.names


## Step 3 - Habilitando CUDA e GPU

In [1]:
%cd /content/drive/MyDrive/detection/TrafficLightDetection/darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

/content/drive/MyDrive/detection/TrafficLightDetection/darknet


## Step 4 - Construindo Darkent

In [2]:
!make

chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:935:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1136:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

## Step 5 - Calculando Novas Âncoras

Dado que foi mudada a resolução padrão do yolov4 é necessário recalcular as 9 âncoras

In [3]:
!./darknet detector calc_anchors data/obj.data -num_of_clusters 9 -width 1280 -height 736

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0

 num_of_clusters = 9, width = 1280, height = 736 
 read labels from 9 images 
 loaded 	 image: 8 	 box: 25
 all loaded. 

 calculating k-means++ ...

 iterations = 2 


counters_per_class = 8, 8, 8, 1

 avg IoU = 89.02 % 

Saving anchors to the file: anchors.txt 
anchors =   5,  8,   6, 13,   7, 12,   7, 14,   9, 17,  12, 25,  16, 32,  29, 68,  33, 69
^C


Com as novas âncoras calculadas, foi alterado no .cfg do github e agora vamos introduzi-lo no darknet

In [3]:
%cd /content/drive/MyDrive/detection/TrafficLightDetection/darknet
!wget https://raw.githubusercontent.com/vilsonrodrigues/ComputerVision/master/ITA_Challenge_2021-TrafficLightDetection/yolov4.cfg
!ls

/content/drive/MyDrive/detection/TrafficLightDetection/darknet
--2021-05-25 19:39:50--  https://raw.githubusercontent.com/vilsonrodrigues/ComputerVision/master/ITA_Challenge_2021-TrafficLightDetection/yolov4.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13338 (13K) [text/plain]
Saving to: ‘yolov4.cfg’

yolov4.cfg          100%[===================>]  13.03K  --.-KB/s    in 0.001s  

2021-05-25 19:39:51 (12.4 MB/s) - ‘yolov4.cfg’ saved [13338/13338]

3rdparty	counters_per_class.txt	include		       results
anchors.txt	darknet			json_mjpeg_streams.sh  scripts
backup		DarknetConfig.cmake.in	libdarknet.so	       src
bad.list	darknet_images.py	LICENSE		       uselib
build		darknet.py		Makefile	       vcpkg.json
build.ps1	darknet_video.py	net_cam_v3.sh	       

## Step 6 - Treinamento

Iniciando treino do zero

In [4]:
!./darknet detector train data/obj.data yolov4.cfg -dont_show -map

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 Prepare additional network for mAP calculation...
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 64, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1   1280 x 736 x   3 -> 1280 x 736 x  32 1.628 BF
   1 conv     64       3 x 3/ 2   1280 x 736 x  32 ->  640 x 368 x  64 8.682 BF
   2 conv     64       1 x 1/ 1    640 x 368 x  64 ->  640 x 368 x  64 1.929 BF
   3 route  1 		                           ->  640 x 368 x  64 
   4 conv     64       1 x 1/ 1    640 x 368 x  64 ->  640 x 368 x  64 1.929 BF
   5 conv     32       1 x 1/ 1    640 x 368 x  64 ->  640 x 368 x  32 0.965 BF
   6 conv     64       3 x 3/ 1    640 x 368 x  32 ->  640 x 368 x  64 8.682 BF
   7 Shortcut Layer: 4,  wt = 0, wn

Para restatar o treino caso a sessão caia, basta reconectar o drive, habilitar o cuda e o opencv e fazer o make para construir o yolov4. Depois executa a célula abaixo

In [ ]:
!./darknet detector train data-detection/obj.data yolov4.cfg backup-weights/yolov4_last.weights -dont_show -map

## Step 7 - Visualizando Métricas

Função auxiliar

In [ ]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

Só funcionará se o treinamento não for interrompido

In [ ]:
imShow('chart.png')

Checando a **mAP (mean average precision)**. Atenção ao selecionar os melhores pesos. Hipoteticamente pode ser na 10000 interação como está abaixo

In [ ]:
%cd backup
!ls

/content/drive/MyDrive/detection/VowelDetectionLibras-yolov4/darknet/backup-weights
yolov4_best.weights  yolov4_last.weights


In [ ]:
!./darknet detector map data-detection/obj.data yolov4.cfg backup/yolov4_best.weights -points 0

 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 610, cudnn_half = 0, GPU: Tesla P4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    224 x 224 x   3 ->  224 x 224 x  32 0.087 BF
   1 conv     64       3 x 3/ 2    224 x 224 x  32 ->  112 x 112 x  64 0.462 BF
   2 conv     64       1 x 1/ 1    112 x 112 x  64 ->  112 x 112 x  64 0.103 BF
   3 route  1 		                           ->  112 x 112 x  64 
   4 conv     64       1 x 1/ 1    112 x 112 x  64 ->  112 x 112 x  64 0.103 BF
   5 conv     32       1 x 1/ 1    112 x 112 x  64 ->  112 x 112 x  32 0.051 BF
   6 conv     64       3 x 3/ 1    112 x 112 x  32 ->  112 x 112 x  64 0.462 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 112 x 112 x  64 0.001 BF
   8 conv  

## Step 8 - Testando o Detector

Primeiro vamos alterar o batch para 1 e as subdivisions para 1 também

In [ ]:
! sed -i 's / batch = 64 / batch = 1 /' yolov4.cfg 
! sed -i 's / subdivisions = 16 / subdivisions = 1 /' yolov4.cfg 

Executando o detector para 1 imagem

In [ ]:
!./darknet detector test data-detection/obj.data yolov4.cfg backup/yolov4_best.weights mask_test_images/image1.jpg -thresh 0.3 
imShow('predictions.jpg')

Rodando em um vídeo

In [ ]:
!./darknet detector demo data-detection/obj.data yolov4.cfg backup-weights/yolov4_best.weights -dont_show mask_test_videos/test3.mp4 -i 0 -out_filename mask_test_videos/finalresult.avi 

Rodando em tempo real na webcam. Maior parte do código é fornecido pelo colab

In [ ]:
# Code from theAIGuysCode Github (https://github.com/theAIGuysCode/YOLOv4-Cloud-Tutorial/blob/master/yolov4_webcam.ipynb)
# Adjusted for my custom YOLOv4 trained weights, config and obj.data files

# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import matplotlib.pyplot as plt
%matplotlib inline


# import darknet functions to perform object detections
from darknet import *
# read architecture
network, class_names, class_colors = load_network("yolov4.cfg", "data-detection/obj.data", "backup-weights/yolov4_best.weights")
width = network_width(network)
height = network_height(network)

# darknet helper function to run detection on image
def darknet_helper(img, width, height):
  darknet_image = make_image(width, height, 3)
  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img_resized = cv2.resize(img_rgb, (width, height),
                              interpolation=cv2.INTER_LINEAR)

  # get image ratios to convert bounding boxes to proper size
  img_height, img_width, _ = img.shape
  width_ratio = img_width/width
  height_ratio = img_height/height

  # run model on darknet style image to get detections
  copy_image_from_bytes(darknet_image, img_resized.tobytes())
  detections = detect_image(network, class_names, darknet_image)
  free_image(darknet_image)
  return detections, width_ratio, height_ratio

# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes  

# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    frame = js_to_image(js_reply["img"])

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # call our darknet helper on video frame
    detections, width_ratio, height_ratio = darknet_helper(frame, width, height)

    # loop through detections and draw them on transparent overlay image
    for label, confidence, bbox in detections:
      left, top, right, bottom = bbox2points(bbox)
      left, top, right, bottom = int(left * width_ratio), int(top * height_ratio), int(right * width_ratio), int(bottom * height_ratio)
      bbox_array = cv2.rectangle(bbox_array, (left, top), (right, bottom), class_colors[label], 2)
      bbox_array = cv2.putText(bbox_array, "{} [{:.2f}]".format(label, float(confidence)),
                        (left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        class_colors[label], 2)

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes   

<IPython.core.display.Javascript object>

Rodando na câmera do seu celular

Baixe o app IP Webcam. Substitua o endereço abaixo, mostrado no aplicativo do telefone e execute:

(falta verificar melhor, apesar do Alexey citar não funcionou por hora)

In [ ]:
./darknet detector demo data-detection/obj.data yolov4.cfg backup-weights/yolov4_best.weights http://10.0.0.103:8080/video?dummy=param.mjpg -i 0